In [1]:
# Original code by Masaki Inaba
# Modified by SCEL Forecasting Team
# Dataset: la Ola (2011) - 1 year, from [9:00am, 5:00pm]


# MAKE SURE YOU CHANGE THE PATH TO THE LA OLA DATA.

In [2]:
# Imports
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


In [3]:
# Functions
def int_24hm_to_min(num):
    hours = num // 100;
    minutes = num % 100;
    return hours*60 + minutes

In [4]:
# Read in data with Pandas
raw_data_df = pd.read_csv("20110101LaOla.csv")

In [5]:
# Preview raw data
raw_data_df.head()

,Unnamed: 0,Year,DOY,HST,Global Horizontal [W/m^2],Direct Normal [W/m^2],Diffuse Horizontal [W/m^2],Air Temperature [deg C],Station Pressure [mBar],Avg Wind Speed @ 3m [m/s],...,Diffuse (uncorrected) [W/m^2],Global (secondary) [W/m^2],GPS Status,Zenith Angle [degrees],Azimuth Angle [degrees],CR1000 Temp [deg C],RSR Battery [VDC],Rel Humidity [%],Latitude [deg],Longitude [deg]
0,0,2011,1,0,0.000000,0.0,0.000000,16.85,972,1.113,...,0.000000,0.109349,-1.0,172.3,251.5,17.77,13.00,98.5,20.7669,-156.923
1,0,2011,1,1,0.080659,0.0,0.080659,16.85,972,0.900,...,0.072632,0.072899,-1.0,172.6,251.0,17.76,13.00,98.5,20.7669,-156.923
2,0,2011,1,2,0.040330,0.0,0.040330,16.85,972,0.988,...,0.036316,0.109350,-1.0,172.8,250.5,17.74,13.00,98.5,20.7669,-156.923
3,0,2011,1,3,0.524285,0.0,0.524285,16.84,972,1.038,...,0.472113,0.656099,0.0,173.0,250.0,17.72,12.97,98.5,20.7669,-156.923
4,0,2011,1,4,0.524286,0.0,0.524286,16.84,972,1.063,...,0.472114,0.473849,0.0,173.2,249.4,17.71,12.96,98.5,20.7669,-156.923


In [6]:
# Select only certain columns that we're interested in.

# Select by column name.
df = raw_data_df.loc[:, ['Year', 'DOY', 'HST', 'Global Horizontal [W/m^2]', 'Zenith Angle [degrees]']]
df.head()

,Year,DOY,HST,Global Horizontal [W/m^2],Zenith Angle [degrees]
0,2011,1,0,0.000000,172.3
1,2011,1,1,0.080659,172.6
2,2011,1,2,0.040330,172.8
3,2011,1,3,0.524285,173.0
4,2011,1,4,0.524286,173.2


In [7]:
# Rename columns to more simple names for easier coding.
df.columns = ['year', 'day', 'time', 'irradiation', 'zenith angle']
df.head()

,year,day,time,irradiation,zenith angle
0,2011,1,0,0.000000,172.3
1,2011,1,1,0.080659,172.6
2,2011,1,2,0.040330,172.8
3,2011,1,3,0.524285,173.0
4,2011,1,4,0.524286,173.2


In [8]:
# Filter by:
# year == 2011,
# between 9:00am and 5:00pm, inclusive both bounds
df_filtered = df[
    df['year'] == 2011
]

df_filtered = df_filtered[
    df_filtered['time'] >= 900
]

df_filtered = df_filtered[
    df_filtered['time'] <= 1700
]

# Preview shape of the DataFrame
df_filtered.shape

(175565, 5)

In [9]:
# Extraction of solar irradiance; reshaping of column into
# (number of minutes, number of days) matrix with numpy.
r = df_filtered['irradiation']

N = 60 * 8 + 1 #the number of sample per day
M = 365
print(N, M, r.shape)

R = np.reshape(r,(N,M),order='F')

481 365 (175565,)


In [10]:
# d = the day axis
d = (df_filtered['day'])

# t = the time axis
# We use our custom converter function to turn the
# "time" column's timestamp values into a raw minute-only value from 0 to N.
t = np.apply_along_axis(int_24hm_to_min, 0, df_filtered['time'])

# Remove duplicate entries for day and time.
d = np.unique(d)
t = np.unique(t)

# Check the shape.
print(t.shape,d.shape,R.shape)

# Prepare the 3D coordinates by creating a meshgrid array.
X,Y = np.meshgrid(d,t)
print(X.shape)
print(Y.shape)


(481,) (365,) (481, 365)
(481, 365)
(481, 365)


In [11]:
#
#
# REGULAR DATA GRAPH
#
#
fig = plt.figure()
ax  = Axes3D(fig)

# Limits
ax.set_xlim(0,370)
ax.set_ylim(500,1000)

# Labels
ax.set_xlabel("Day of Year [day]")
ax.set_ylabel("Time of Day [minutes]")
ax.set_zlabel("Solar Irradiation [W/m^2]")

# Plot 3D points from meshgrids and data points.
# Note: Each meshgrid accounts for only 1 axes (e.g. X contains only the X-coordinates for each point.)
ax.plot_surface(X, Y, R, rstride=30, cstride=30, cmap=cm.coolwarm)
plt.show()

In [13]:
#
#
# NORMALIZED DATA GRAPH
#
#

# Filter according to cos of zenith angle.
df_filtered['zenith angle'] = np.cos(np.deg2rad(df_filtered['zenith angle']))
z = df_filtered['irradiation']/df_filtered['zenith angle']

# Once again, reshape the data.
Z = np.reshape(z,(N,M),order='F')

# Using Axes3D-based plot.
fig = plt.figure()
ax  = Axes3D(fig)

# Limits
ax.set_xlim(0,370)
ax.set_ylim(500,1000)

# Labels
ax.set_xlabel("Day of Year [day]")
ax.set_ylabel("Time of Day [minutes]")
ax.set_zlabel("Zenith Angle-Normalized Solar Irradiance [W/m^2]")

# Plot 3D points from meshgrid arrays and data points.
ax.plot_surface(X, Y, Z, rstride=30, cstride=30, cmap=cm.coolwarm)
plt.show()